#### Efetuando Extração de texto do Youtube

In [1]:
# !pip install selenium webdriver-manager

In [2]:
import json
import io
import time
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def scrape_youtube_comments(url):
    """
    Extrai os comentários do vídeo do YouTube fornecido pela URL
    e salva o resultado em 'comentarios_extraidos.json'.
    
    Args:
        url (str): URL do vídeo do YouTube.
    """
    # Configuração do Chrome com modo headless
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    driver.get(url)
    time.sleep(5)

    try:
        title = driver.find_element(By.XPATH, '//*[@id="container"]/h1/yt-formatted-string').text
        comment_section = driver.find_element(By.XPATH, '//*[@id="comments"]')
    except exceptions.NoSuchElementException:
        print("❌ Erro: Verifique os seletores ou aguarde mais tempo para carregar os elementos.")
        driver.quit()
        return

    driver.execute_script("arguments[0].scrollIntoView();", comment_section)
    time.sleep(7)

    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    try:
        username_elems = driver.find_elements(By.XPATH, '//*[@id="author-text"]')
        comment_elems = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
    except exceptions.NoSuchElementException:
        print("❌ Erro: Não foi possível encontrar os elementos dos comentários.")
        driver.quit()
        return

    print(f"\n🎬 TÍTULO DO VÍDEO: {title}\n")

    comentarios_extraidos = []
    for username, comment in zip(username_elems, comment_elems):
        comentarios_extraidos.append({
            "usuario": username.text.strip(),
            "comentario": comment.text.strip()
        })

    with io.open("comentarios_extraidos.json", "w", encoding="utf-8") as json_file:
        json.dump(comentarios_extraidos, json_file, indent=4, ensure_ascii=False)

    print(f"✅ {len(comentarios_extraidos)} comentários salvos em 'comentarios_extraidos.json'")
    driver.quit()


# === CELULAR DE EXECUÇÃO NO JUPYTER ===
# Você pode colar isso numa célula e executar:
video_url = input("🔗 Cole a URL do vídeo do YouTube: ").strip()
scrape_youtube_comments(video_url)
# === FIM DO CÓDIGO ===


🎬 TÍTULO DO VÍDEO: 

✅ 704 comentários salvos em 'comentarios_extraidos.json'


#### Instalação das Bibliotecas

In [3]:
# !pip install transformers

In [4]:
# !pip install torch

In [5]:
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# !pip install transformers
# !pip install ipywidgets tqdm
# jupyter nbextension enable --py widgetsnbextension --sys-prefix

#### Análise de Sentimentos dos Comentários Extraídos

In [6]:
import json
from transformers import pipeline
from tqdm import tqdm  # barra de progresso (opcional, mas útil)

# 1. Carrega os comentários do JSON
with open("comentarios_extraidos.json", "r", encoding="utf-8") as f:
    comentarios = json.load(f)

# 2. Inicializa o pipeline de análise de sentimentos
sentiment_pipe = pipeline("text-classification", model="Adilmar/caramelo-smile-2")

# 3. Analisa cada comentário
comentarios_com_sentimento = []

for item in tqdm(comentarios, desc="Analisando sentimentos"):
    texto = item["comentario"]
    resultado = sentiment_pipe(texto[:512])  # Limita a 512 tokens para evitar erro
    item["sentimento"] = resultado[0]["label"]
    item["score"] = resultado[0]["score"]
    comentarios_com_sentimento.append(item)

# 4. Salva os resultados em um novo JSON
with open("comentarios_com_sentimento.json", "w", encoding="utf-8") as f:
    json.dump(comentarios_com_sentimento, f, ensure_ascii=False, indent=4)

# 5. (Opcional) Visualiza os 5 primeiros com sentimentos
for c in comentarios_com_sentimento[:5]:
    print(f"Usuário: {c['usuario']}\nComentário: {c['comentario']}\nSentimento: {c['sentimento']} (score: {c['score']:.2f})\n")


Device set to use cpu
Analisando sentimentos: 100%|██████████| 704/704 [02:10<00:00,  5.38it/s]

Usuário: @Deanpy
Comentário: 1:43  O Jonathan kent voltou a ser um homem que apoia seu filho a fazer o melhor, finalmente!!
Sentimento: positive (score: 1.00)

Usuário: @FabiojuniorPereiradossan-qx4vk
Comentário: Isso que é filme 


Hypado para o filme, o Superman voltou.
Sentimento: neutral (score: 0.89)

Usuário: @claudioeduardo5878
Comentário: Só pela trilha sonora clássica já fiquei muito na expectativa
Sentimento: positive (score: 0.98)

Usuário: @giovannacosta2773
Comentário: Detalhe, o superman nao precisa de ajuda pra se levantar, porém, ele aceitou a ajuda de um humano. Amei!
Sentimento: positive (score: 0.99)

Usuário: @andreflorenciotorres349
Comentário: Parece que, depois de muitos anos, vou ao cinema assistir a um genuíno filme de super-heroi. O maior de todos. E desta vez, com todas as cores.
Sentimento: positive (score: 0.99)



### Analisado de Sentimentos - Testando o modelo abaixo + 100M de parametros em Pt-Br
https://huggingface.co/Adilmar/caramelo-smile-2

In [7]:
# from transformers import pipeline

# # Load the classification pipeline with the specified model
# # pipe = pipeline("text-classification", model="tabularisai/multilingual-sentiment-analysis")
# pipe = pipeline("text-classification", model="Adilmar/caramelo-smile-2")

# # Classify a new sentence
# sentence = "o brasil não é o pra amadores"
# result = pipe(sentence)

# # Print the result
# print(result)
